# Programmieraufgabe 4: QR-Algorithmus für reelle Matrizen

**Abgabe in den Programmiertutorien am 2./3. Juli 2025.**

In diesem Notebook erweitern wir den QR-Algorithmus aus Programmieraufgabe 3 so, dass er auch mit reellen Matrizen mit komplex konjugierten Eigenwerten zurecht kommt und dabei nur reelle Operationen benutzt. Dazu verwenden wir die "bulge chasing" Technik.

Die Aufgabe erscheint auf den ersten Blick vielleicht etwas lang. Das liegt aber nur daran, dass die zu implementierende Vorgehensweise nochmal im Detail erklärt wird und einige Code-Bausteine, insbesondere zum Testen von Prozeduren, vorgebeben sind.  

In der Programmieraufgabe werden Sie die Prozeduren `hess` und `qr_alg_shift` aus den Teilen (e) bzw. (h) des Notebooks aus Programmieraufgabe 3 wiederverwenden können. Sie können sich diese beiden Prozeduren entweder aus Ihrer eigenen Lösung oder aus dem im Ilias bereit gestellten Lösungsvorschlag (ab 24. Juni verfügbar) an geeigneter Stelle kopieren.

Wir benötigten in diesem Notebook die folgenden Module:

In [4]:
import numpy as np
import scipy.linalg as spla  # für Matrixzerlegungen und co
import numpy.random as rnd   # für alles, was mit Zufallszahlen zu tun hat

Außerdem können wieder die beiden Prozeduren `printvector` und `printmatrix` zur Ausgabe von Vektoren und Matrizen verwendet werden.

In [5]:
def printvector(v):
    if v.dtype == "int":
        print(''.join([' {:4}'.format(item) for item in v])+"\n")
    elif v.dtype == "complex128":
        print(''.join([' {:16.3f}'.format(item) for item in v])+"\n")
    else:
        print(''.join([' {:7.3f}'.format(item) for item in v])+"\n")

In [6]:
def printmatrix(A):
    if A.dtype == "int":
        print('\n'.join([''.join(['  {:4}'.format(item) for item in row]) for row in A])+"\n")
    elif A.dtype == "complex128":
        print('\n'.join([''.join(['  {:16.3f}'.format(item) for item in row]) for row in A])+"\n")   
    else:
        print('\n'.join([''.join(['  {:7.3f}'.format(item) for item in row]) for row in A])+"\n")       

# Problemstellung & Modellmatrix

Wir betrachten als Modellmatrix zunächst die Matrix $A = A_6$ aus dem Notebook zu Programmieraufgabe 3. Diese wurde durch eine Ähnlichkeitstransformation aus der Blockdiagonalmatrix 
$$
D_6 = \begin{pmatrix} 2 \\ & 1 \\ && 1 & -1 \\ && 1 & 1 \\ &&&& \frac12 \end{pmatrix}
$$
erzeugt, d.h. $A = S^{-1} D_6 S$ für eine invertierbare Matrix $S$. Der $2\times2$ Diagonalblock hat das komplex konjugierte Eigenwertpaar $1\pm\mathrm{i}$. Dasselbe gilt dann offensichtlich auch für die Matrizen $D_6$ bzw. $A$.

Zunächst erzeugen wir die Matrix $A$ wie im letzten Notebook und bringen sie mit der Prozedur `hess` in Hessenberg-Form:

<!-- Am Ende des Notebooks wollen wir eine auf dem QR-Algorithmus basierende Prozedur haben, die auch von dieser Matrix $A$ alle Eigenwerte berechnen kann, und dabei alle eigentlichen Iterationen in reeller Arithmetik durchführt. -->


In [ ]:
def hess(A):
    n = np.size(A,0) # Anzahl Zeilen/Spalten
    for j in range(n-2):
        # Spiegele erste Spalte von A[j+1:,j:] auf alpha-faches von e_1
        
        # Baue passenden Householder-Vektor zusammen
        x = np.copy(A[j+1:,j]) # Erste Spalte von A[j+1:,j:]
        alpha = - x[0]/np.abs(x[0]) * np.linalg.norm(x)
        v = x
        v[0] -= alpha
        v = v/np.linalg.norm(v)
        
        # Wende Q = I - 2vv^H von links an
        A[j+1,j] = alpha
        A[j+2:,j] = 0
        A[j+1:,j+1:] += np.outer( (-2*v) , v.conj() @ A[j+1:,j+1:] )

        # Wende Q = I - 2vv^H von rechts an
        A[:,j+1:] -= np.outer( A[:,j+1:] @ (2*v) , v.conj() )

    return A

In [8]:
# (Block-)Diagonalmatrix
A = np.array([[2,0,0,0,0],[0,1,0,0,0],[0,0,1,-1,0],[0,0,1,1,0],[0,0,0,0,1/2]])

# Ähnlichkeitstransformation
S = np.array([
    [2, -1, 1, 0, -1],
    [-1, 1, 2, 2, 2],
    [-1, 0, 2, -1, 1],
    [-1, 2, 2, 2, 1],
    [2, -1, 2, 0, -1]
])
S_inv = spla.inv(S)

# Finale Matrix A
A = S_inv @ A @ S

# Matrix A in Hessenberg-Form
A_hess = hess(A.copy())

printmatrix(A_hess)

   16.500   11.093   -2.337   10.916    0.317
  -20.700  -13.821    2.699  -15.152    0.446
    0.000    0.063    1.121   -1.915   -0.454
    0.000    0.000    0.626    0.711   -1.337
    0.000    0.000    0.000   -0.205    0.989



Wir arbeiten uns schrittweise in Richtung des finalen Programms vor:

# Schritt 1: Bulge chasing
Gegeben sei hier zunächst eine Matrix $A\in\mathbb{R}^{n\times n}$ mit der Struktur
$$
A = \begin{pmatrix}
    * & * & * & * & \cdots & * \\
    * & * & * & * & & \vdots \\
    \otimes & * & * & * & & \\
    \otimes & \otimes & * & * & & \\
     & & & \ddots & \ddots & \\
     & & & & * & *
    \end{pmatrix}.
$$
Die drei mit $\otimes$ markierten Einträge bilden den _bulge_. Wenn dieser bulge nicht wäre, dann hätte die Matrix Hessenberg-Form.

**(a) Ändern Sie die Prozedur `hess` aus Programmieraufgabe 3 zu einer Prozedur `bulge_chasing` ab, die eine Matrix $A$ mit der obigen Struktur durch Householder-Transformationen auf Hessenberg-Form bringt (siehe Schritt 4 aus dem Francis' QR-Schritt). Achten Sie dabei darauf, in jedem Schritt nur die Zeilen/Spalten der Matrix zu bearbeiten, bei denen dies nötig ist.**

In [55]:
def bulge_chasing(A):
    n = np.size(A,0) # Anzahl Zeilen/Spalten
    for j in range(n-2):
        # Spiegele erste Spalte von A[j+1:,j:] auf alpha-faches von e_1

        is_last_bulge = (j == n-3)  # Check if we are at the last bulge
        edge_case_modifier = 0 if not is_last_bulge else -1

        # Baue passenden Householder-Vektor zusammen
        A_bulge_block = A[j+1:j+4, j:(j+3 + edge_case_modifier)]  # 3x3 Block

        x = np.copy(A_bulge_block[:,0]) # first column of the bulge block
        alpha = - x[0]/np.abs(x[0]) * np.linalg.norm(x)
        v = x
        v[0] -= alpha
        v = v/np.linalg.norm(v)

        Householder_block = np.eye(3 + edge_case_modifier) - 2 * np.outer(v, v.conj())

        # transform matrix A with the Householder transformation
        if not is_last_bulge:
          T = spla.block_diag(np.eye(j+1), Householder_block, np.eye(n-j-4))
        else:
          T = spla.block_diag(np.eye(j+1), Householder_block)


        # printmatrix(T)

        A = T @ A @ T

        # printmatrix(A)
        
        # # Wende Q = I - 2vv^H von links an
        # A[j+1,j] = alpha
        # A[j+2:,j] = 0
        # A[j+1:,j+1:] += np.outer( (-2*v) , v.conj() @ A[j+1:,j+1:] )

        # # Wende Q = I - 2vv^H von rechts an
        # A[:,j+1:] -= np.outer( A[:,j+1:] @ (2*v) , v.conj() )

    return A

printmatrix(bulge_chasing(A_hess.copy()))

   16.500  -11.093    2.337  -10.916    0.317
   20.700  -13.821    2.699  -15.152   -0.446
    0.000    0.063    1.121   -1.915    0.454
    0.000    0.000    0.626    0.711    1.337
    0.000    0.000    0.000    0.205    0.989



Sie können die Prozedur mit folgemdem Code testen. Im ersten Code-Block wird eine zufällige Hessenberg-Matrix mit bulge erstellt und deren Eigenwerte angezeigt. Im zweiten Block wird dann die Prozedur `bulge_chasing` angewandt und die Ergebnismatrix sowie deren Eigenwerte ausgegeben. Hat die Ergebnismatrix die gewünschte Hessenburg-Struktur? Und hat sie weiterhin dieselben Eigenwerte wie die Ausgangsmatrix?

In [56]:
# Beispielmatrix
n = 7
H_start = -1 + 2*rnd.rand(n,n)
H_start = np.triu(H_start,-1)
H_start[2,0] = rnd.random(); H_start[3,0] = rnd.random(); H_start[3,1] = rnd.random()
printmatrix(H_start)

   -0.941   -0.965    0.895    0.427    0.387    0.335    0.958
   -0.759   -0.372    0.535   -0.805   -0.989   -0.132    0.865
    0.628   -0.950    0.665   -0.876   -0.259   -0.454    0.816
    0.195    0.560    0.371    0.994    0.200    0.499    0.971
    0.000    0.000    0.000    0.516   -0.690    0.643   -0.756
    0.000    0.000    0.000    0.000    0.157    0.599   -0.017
    0.000    0.000    0.000    0.000    0.000   -0.051    0.327



In [57]:
print('Eigenwerte vorher:')
printvector(spla.eigvals(H_start))

Eigenwerte vorher:
    -1.641+0.000j    -0.916+0.000j     0.490+0.991j     0.490-0.991j     1.193+0.000j     0.650+0.000j     0.315+0.000j



In [58]:
H_new = bulge_chasing(H_start.copy())
printmatrix(H_new)

print('Eigenwerte nachher:')
printvector(spla.eigvals(H_new))

   -0.941    1.372   -0.103   -0.120   -0.387   -0.349    0.957
    1.004    0.256    0.015   -0.112   -0.654    0.068    0.047
    0.000   -1.042   -0.012   -1.272   -1.076   -0.395   -1.055
   -0.000    0.000    0.633    0.846    0.175    0.409   -1.144
   -0.000    0.000   -0.000    0.865   -0.509    0.690    0.709
    0.000    0.000   -0.000    0.000    0.115    0.616    0.069
    0.000   -0.000    0.000    0.000   -0.000    0.051    0.326

Eigenwerte nachher:
    -1.641+0.000j    -0.916+0.000j     0.490+0.991j     0.490-0.991j     1.193+0.000j     0.650+0.000j     0.315+0.000j



# Schritt 2: Lemma 6.28 in Action

Nehmen wir an, dass wir nach einer gewissen Anzahl $k$ an Schritten des QR-Algorithmus eine *reellwertige* Hessenberg-Matrix $H_k$ erhalten haben. Dann schauen wir uns zwei weitere Schritte des QR-Algorithmus an, wobei die Shifts $\mu_k$ und $\mu_{k+1}$ entweder beide reell seien, oder ein komplex konjugiertes Paar bilden (also $\mu_{k+1} = \overline{\mu_k}$):
$$
\begin{align*}
H_k - \mu_k I &= Q_k R_k && \text{(QR-Zerlegung)}
\\
H_{k+1} &= R_k Q_k + \mu_k I
\\
H_{k+1} - \mu_{k+1} I &= Q_{k+1} R_{k+1} && \text{(QR-Zerlegung)}
\\
H_{k+2} &= R_{k+1} Q_{k+1} + \mu_{k+1} I
\end{align*}
$$
In der Vorlesung wurde diskutiert, dass 
* $H_{k+2} = Q^H H_k Q$ mit $Q = Q_k Q_{k+1}$ gilt,
* $Q$ und $H_{k+2}$ wieder reelle Matrizen sind, wenn die QR-Zerlegungen so gewählt werden, dass die Diagonalelemente von $R_k$ und $R_{k+1}$ reell sind,
* die Matrix $Q$ zu einer reellwertigen QR-Zerlegung der reellwertigen Matrix $M = H_k^2 - (\mu_k+\mu_{k+1}) H_k + \mu_k \mu_{k+1} I$ gehört (siehe auch Hausaufgabe 11). 

Die Zwischenergebnisse, also die Matrizen $Q_k,Q_{k+1},R_k,R_{k+1}$ und $H_{k+1}$ werden aber im Falle komplexer Shifts dennoch komplexe Einträge haben, sodass in komplexer Arithmetik gerechnet werden muss.

Daher wurde eine zweite Variante diskutiert, mit der auf ganz anderem Wege dasselbe Ergebnis (bis auf Vorzeichen) erzielt wird:
* Wende diejenige (reelle) Householder-Transformation $T_0$, welche die erste Spalte von $M$ auf ein Vielfaches von $e_1$ spiegelt, von links und rechts auf $H_k$ an,
* Transformiere die resultierende Matrix durch bulge chasing (also durch Anwendung geeigneter, reeller Householder-Transformationen $T_1,...,T_{n-2}$) auf Hessenberg-Form. Erhalte Matrix $\widehat{H} = T^\top H_k T$ mit $T = T_0 T_1 \cdots T_{n-2}$.
* Die Matrix $T$ hat (evtl. bis auf das Vorzeichen) dieselbe erste Spalte wie $Q$. Nach Lemma 6.28 stimmen daher sowohl $\widehat{H}$ und $H_{k+2}$ (bis auf Vorzeichen) als auch $T$ und $Q$ (bis auf unterschiedliche Vorzeichen der Spalten) überein. Insbesondere stammt auch $T$ aus einer reellwertigen QR-Zerlegung von $M$, sodass man im QR-Algorithmus genausogut mit der Matrix $\widehat{H} = T^\top H_k T$ statt mit $H_{k+2} = Q^H H_k Q = Q^\top H_k Q$ weiter machen kann.

Der entscheidende Vorteile der zweiten Vorgehensweise ist, dass hier (auch bei komplex konjugierten Shifts) nur reelle Rechenoperationen nötig sind!

Die Matrizen $\widehat{H}$ bzw. $H_{k+2}$ sind übrigens die einzigen Größen, die man für die weiteren Schritte des QR-Algorithmus braucht. Die Matrizen $T$ bzw. $Q$ braucht man sich nicht merken.

**(b) Schreiben Sie eine Prozedur `double_step`, welche die zweite Vorgehensweise umsetzt, d.h. welche**
* **eine Hessenberg-Matrix $H$ und zwei Shifts $\mu_1$,$\mu_2$ (entweder beide reell oder ein komplex konjugiertes Paar) als Eingabe erhält,**
* **die erste Spalte der Matrix $M = H^2 - (\mu_1 + \mu_2) H + \mu_1\mu_2 I$ berechnet (ohne Matrixprodukte!),**
* **die Householder-Transformation, die die erste Spalte von $M$ auf ein Vielfaches von $e_1$ spiegelt, von links und rechts auf $H$ anwendet,**
* **das Ergebnis mittels der Prozedur `bulge_chasing` wieder auf eine Hessenberg-Matrix $\widehat{H}$ transformiert,**
* **und schließlich die Matrix $\widehat{H}$ zurück gibt.**

Hinweise:
* Für beliebige $\mu_1,\mu_2\in\mathbb{C}$ sind natürlich auch $\mu_1+\mu_2, \mu_1\mu_2 \in \mathbb{C}$, sodass auch `numpy` für das Ergebnis komplexe Zahlen als Datentyp verwendet. Wir lassen ja aber nur komplex konjugierte Shifts $\mu_1 = \overline{\mu_2}$ zu, für die $\mu_1+\mu_2 = 2 \mathrm{real}(\mu_1)$ und $\mu_1\mu_2 = |\mu_1|^2$ reell sind. Wenden Sie daher nach Berechnung von $\mu_1+\mu_2$ und $\mu_1\mu_2$ jeweils den Befehl `np.real(...)` an, um den Imaginärteil (der eh Null ist) weg zu lassen und wieder eine Variable mit reellem Datentyp zu erhalten.
* Geeigneter Code zum Testen der Prozedur steht unten bereit.

In [ ]:
def double_step(H, shift_1, shift_2):
    n = np.size(H, 0)  # Anzahl Zeilen/Spalten
    shift_sum=np.real(shift_1 + shift_2)
    shift_product=np.real(shift_1 * shift_2)
    H_first_col = H[:,0]
    M_column = H @ H_first_col - (shift_sum * H_first_col)
    M_column[0] += shift_product

    x = M_column
    alpha = - x[0]/np.abs(x[0]) * np.linalg.norm(x)
    v = x
    v[0] -= alpha
    v = v/np.linalg.norm(v)

    Householder_matrix = np.eye(n) - 2 * np.outer(v, v.conj())

    H = Householder_matrix @ H @ Householder_matrix

    return bulge_chasing(H)

Mit dem folgenden Code können Sie Ihre Prozedur wieder testen. Darin wird zunächst eine zufällige Hessenberg-Matrix $H$ erstellt sowie zwei beliebige reelle Shifts oder ein komplex konugiertes Shift-Paar $\mu_1,\mu_2$ gewählt. Dann wird einerseits das Ergebnis der Prozedur `double_step` ausgegeben, und andererseits das Ergebnis, wenn auf klassische Art zwei QR-Algorithmus-Schritte mit Shifts $\mu_1,\mu_2$ durchgeführt werden (beachten Sie, dass der `qr` Befehl aus `scipy` tatsächlich die QR-Zerlegungen komplexer Matrizen so wählt, dass die Diagonaleinträge von $R$ reell sind). Nach der Diskussion oben/aus der Vorlesung sollten beide Ergebnisse bis auf Vorzeichen übereinstimmen.

In [62]:
# Zufällige Hessenberg-Matrix mit Einträgen in [-1,1]:
n = 7
H_start = -1 + 2*rnd.rand(n,n)
H_start = np.triu(H_start,-1)

# Shifts:
mu1 = 2+3j; mu2 = 2-3j # komplex konjugierte Shifts
# mu1 = 2; mu2 = 3 # reelle Shifts

# Zwei klassische QR-Schritte:
H1 = H_start.copy()
Q,R = spla.qr(H1-mu1*np.eye(n))
H1 = R@Q + mu1*np.eye(n)
Q,R = spla.qr(H1-mu2*np.eye(n))
H1 = R@Q + mu2*np.eye(n)
print('Ergebnis nach zwei klassischen QR-Schritten:')
printmatrix(H1)

# Prozedur double_step:
H2 = H_start.copy()
H2 = double_step(H2,mu1,mu2)
print('Ergebnis nach Francis QR-Schritt:')
printmatrix(H2)

Ergebnis nach zwei klassischen QR-Schritten:
     -0.621+0.000j      0.700+0.000j     -0.966+0.000j     -0.373+0.000j      0.937+0.000j      0.217-0.000j     -0.202+0.000j
     -0.498+0.000j      0.344+0.000j     -0.752-0.000j     -0.928-0.000j      0.834+0.000j      0.144+0.000j      0.066+0.000j
      0.000+0.000j     -0.514-0.000j      0.812+0.000j     -0.587-0.000j     -0.287+0.000j     -0.194-0.000j      0.576+0.000j
      0.000+0.000j      0.000+0.000j     -0.002+0.000j      0.013+0.000j      0.617+0.000j      1.068-0.000j      0.747-0.000j
      0.000+0.000j      0.000+0.000j      0.000+0.000j     -0.365+0.000j      0.419+0.000j      1.089-0.000j      0.051-0.000j
      0.000+0.000j      0.000+0.000j      0.000+0.000j      0.000+0.000j      1.074-0.000j     -0.066+0.000j      0.574-0.000j
      0.000+0.000j      0.000+0.000j      0.000+0.000j      0.000+0.000j      0.000+0.000j      0.612+0.000j     -0.007+0.000j

Ergebnis nach Francis QR-Schritt:
   -0.621    0.700   -0.966   -

# Schritt 3: QR-Algorithmus für reellwertige Matrizen

Nun wollen wir eine Prozedur schreiben, die den QR-Algorithmus auf reellwertige Matrizen anwendet. Im Wesentlichen werden dafür einfach iterativ immer wieder zwei QR-Algorithmus-Schritte mittels der Prozedur `double_step` durchgeführt. Zwei Fragen sind allerdings noch offen:
* Wie wählen wir jeweils die Shifts $\mu_1$ und $\mu_2$?
* Wann brechen wir den Algorithmus ab (z.B. in dem Sinne, dass wir das Problem auf eine kleinere Matrix reduzieren können)?

Dazu fahren wir eine ähnliche Strategie wie beim QR-Algorithmus für komplexwertige Matrizen aus Programmieraufgabe 3, arbeiten allerdings mit dem $2\times2$-Block $\widetilde{H}$ rechts unten in der Matrix $H$, also mit
$$
    \widetilde{H} = \begin{pmatrix} h_{n-1,n-1} & h_{n-1,n} \\ h_{n,n-1} & h_{n,n} \end{pmatrix}
$$
anstatt nur mit dem letzten Element $h_{n,n}$. In jedem Schritt bestimmen wir die beiden Eigenwerte dieses $2\times2$-Blocks analytisch und verwenden diese als Shifts. Da auch $\widetilde{H}$ reelle Einträge hat, sind die beiden Eigenwerte, und somit die Shifts, wie gefordert entweder beide reell oder ein komplex konjugiertes Paar. 

Die Hoffnung ist, dass wir mit diesen beiden Shifts zwei Eigenwerte der Matrix $H$ nach wenigen Schritten gut approximieren und somit den $2\times2$-Block am Ende abkopppeln können. Dazu müssen wir das Element $h_{n-1,n-2}$ beobachten. Wenn dieses Element (quasi) Null ist, ist die Matrix reduzibel. Die Eigenwerte des $2\times2$ Blocks können wir leicht berechnen (machen wir ja auch ständig für die Shifts). Die Eigenwerte der verbleibenden $(n-2)\times(n-2)$-Matrix berechnen wir, indem wir die Prozedur rekursiv aufrufen. Konkret teilen wir die Matrix auf, wenn
$$ 
|h_{n-1,n-2}| \leq \texttt{eps} \left( |h_{n-2,n-2,}| + |h_{n-1,n-1}| \right)
$$
mit der Maschinengenauigkeit $\texttt{eps}$ gilt.

Zusätzlich behalten wir auch das alte Abbruchkriterium
$$
|h_{n,n-1}| \leq \texttt{eps} \left( |h_{n-1,n-1,}| + |h_{n,n}| \right)
$$
bei, falls eine Abkopplung nur des letzten Elements stattfindet.

**(c) Ändern Sie die Prozedur `qr_alg_shift` aus Programmieraufgabe 3 (h) so zu einer Prozedur `qr_alg_real` ab, dass sie die oben beschriebene Strategie umsetzt. Dazu sollten Sie insbesondere folgendes ändern:**
* **Ist die eingegebene Matrix $H$ eine $2\times2$-Matrix, dann sollten die Prozedur (wie auch bisher schon im $1\times1$-Fall) einfach die beiden Eigenwerte von $H$ zurück geben.**
* **Für $n\not\in\{1,2\}$ müssen in jedem Schritt zunächst die beiden Shifts berechnet werden. Um die eigentlichen zwei QR-Schritte durchzuführen, muss dann nur noch die Prozedur `double_step` angewandt werden.**
* **Das neue Abbruchkritierum sowie sinnvolle Anweisungen für den Fall des Abbruchs müssen ergänzt werden. Beachten Sie beim rekursiven Aufruf der Prozedur (insbesondere im alten Abbruchkriterium), dass Sie den richtigen, neuen Namen der Prozedur verwenden.**

Zur analytischen Berechnung von Eigenwerten reeller $2\times2$-Matrizen können Sie die bereitgestellte Prozedur `ew_2x2` verwenden. Diese liefert einen Vektor aus beiden Eigenwerten zurück (und zwar vom Datentyp "reelle Zahlen", falls die Eigenwerte reell sind, und vom Typ "komplexe Zahl", falls es sich um ein komplex konjugiertes Paar handelt).

In [63]:
def ew_2x2(A):
    det_A = A[0,0]*A[1,1] - A[1,0]*A[0,1]
    spur_A = A[0,0] + A[1,1]
    diskriminante = spur_A**2 - 4*det_A
    if diskriminante < 0:
        lam = ( spur_A + np.array([1j,-1j]) * np.sqrt(-diskriminante) )/2
    else:
        lam = ( spur_A + np.array([1,-1]) * np.sqrt(diskriminante) )/2

    return lam

In [125]:
def qr_alg_real(A, kMax=100):
    H = A.copy()
    n = H.shape[0]
    print('Size: ', n)
    eigvals = np.array([], dtype=H.dtype)
    if n == 2:
        return np.append(eigvals, ew_2x2(H))
    if n == 1:
        return np.append(eigvals, H[0, 0])
    for k in range(kMax):
        shifts = ew_2x2(H[n-2:n, n-2:n])
        H = double_step(H, shifts[0], shifts[1])
        if spla.norm(H[-2, -3]) <= np.finfo(np.float64).eps * (spla.norm(H[-3, -3]) + spla.norm(H[-2, -2])):
            eigvals = np.append(eigvals, shifts)
            eigvals = np.append(eigvals, qr_alg_real(H[:n-2, :n-2], kMax))
            return eigvals
        if spla.norm(H[-1, -2]) <= np.finfo(np.float64).eps * (spla.norm(H[-2, -2]) + spla.norm(H[-1, -1])):
            eigvals = np.append(eigvals, H[n-1, n-1])
            eigvals = np.append(eigvals, qr_alg_real(H[:n-1, :n-1], kMax))
            return eigvals
    print('Warning: Maximum number of iterations reached without convergence.')
    return eigvals

**(d) Führen Sie den folgenden Code-Block aus, um Ihre Prozedur auf die (in Hessenberg-Form gebrachte) Matrix $A$ von ganz oben anzuwenden und somit die Funktionalität Ihres Codes zu überprüfen.**

In [119]:
# print('Eigenwerte laut Python:')
# printvector(np.sort(spla.eigvals(A_hess)))

# lam = qr_alg_real(np.copy(A_hess),20,True)
lam = qr_alg_real(np.copy(A_hess),20)
print('Berechnete Eigenwerte:')
printvector(np.sort(lam))

Size:  5
Size:  3
Size:  1
Berechnete Eigenwerte:
     0.500+0.000j     1.000+0.000j     1.000-1.000j     1.000+1.000j     2.000+0.000j



Wie in Programmieraufgabe 3 können wir mit dieser Prozedur jetzt auch die Eigenwerte beliebiger anderer reellwertiger Matrizen (egal ob mit komplexen Eigenwertpaaren oder nicht) berechnen. Hier zum Beispiel für eine zufällige Matrix: 

In [120]:
n = 15
B = -1 + 2*rnd.rand(n,n)
B_hess = hess(B)

In [156]:
n = 15
B = -1 + 2*rnd.rand(n,n)
B_hess = hess(B)

print('Eigenwerte laut Python:')
printvector(np.sort(spla.eigvals(B_hess)))

print('Francis QR-Algorithmus:')
# lam = qr_alg_real(B_hess.copy(),100,False)
lam = qr_alg_real(B_hess.copy(),100)
print('\nEigenwerte:')
printvector(np.sort(lam))

# Fragen:
# 1. Warum konvergiert mein Algorithmus so langsam?
# 2. Was soll der Parameter boolean machen?

Eigenwerte laut Python:
    -1.734-0.512j    -1.734+0.512j    -1.216+0.000j    -0.760-1.497j    -0.760+1.497j    -0.557+0.000j    -0.433-1.104j    -0.433+1.104j    -0.216-1.519j    -0.216+1.519j     0.763+0.000j     1.220-1.606j     1.220+1.606j     1.802-0.102j     1.802+0.102j

Francis QR-Algorithmus:
Size:  15
Size:  14
Size:  13
Size:  11
Size:  9
Size:  7
Size:  6
Size:  4
Size:  2

Eigenwerte:
    -1.734-0.512j    -1.734+0.512j    -1.216+0.000j    -0.760-1.497j    -0.760+1.497j    -0.557+0.000j    -0.433-1.104j    -0.433+1.104j    -0.216-1.519j    -0.216+1.519j     0.763+0.000j     1.220-1.606j     1.220+1.606j     1.802-0.102j     1.802+0.102j

